# Configuration

Configuration allow to change the behavior of program without change the code or it's binaries.

Users of the configuration are going to be other engineers.

## Designing configuration

When designing configuration, you should consider such questions:
- What should be configurable?
- How granular the configuration should be?
- Can the application work with some default values?
- Is the configuration self contained?

In the perfect programs should not require any configuration as they would figure how how to configure themselves.

If you are implementing the configuration, then be sure about the reasoning why are you investing the effort to make something configurable.

### What to configure

Examples of things that you want to configure:
- Database connection strings.
- Endpoints of external APIs.
- Credentials to connect to external services.
- What external provider of service to use.
- And more...

On the same note you could reason that there are something that you don't need to be able to configure, like:

- Background color of application.
- Button sizes.
- Color of console output.
- And more...

What is worth to configure comes down to correctly identifying what will likely need to be changed between the deployments or naturally overtime.

Making something configurable is going to take more effort, than not to make it configurable. On the same note, implementing configuration is going to result in more code than not implementing configuration.

This reduces the decision wether to make something configurable to:
>Is the time spend developing configuration and increased code complexity worth the benefits this configuration brings?

### Spectrum: customizable vs out-of-the-box

There are trade offs to make when designing what will be configurable:
- Configuration can be made very granular, so even the buttons sizes could be configured, but that would potentially overload the user.
- Configuration can be made very coarse, but that would reduce the opportunities for program reuse and code changes might be requested more often than they could.

You could think about these 2 options being on the opposite sides of the spectrum. Configurations for most the programs will fall somewhere in the middle of that spectrum.

### Mandatory vs optional

Configuration should use sensible default wherever possible and not require user to explicitly specify values.

This helps to reduce the mental load of user to launch the application, and thus reduces friction for it to be used.

### Configuration should be hermetic

Configuration should be self contained.

Configuration should not depend on any other external sources that it does not control. 

## Configuration in ASP.NET

Because ASP.NET currently runs as generic-host application, the configuration happens near the entry point of the program.

Web application builder exposes the configuration manager class that can be used to setup the configuration:
```csharp
var builder = WebApplication.CreateBuilder(args);
ConfigurationManager configurationManager = builder.Configuration;
```

This `ConfigurationManager` can be used to setup different providers and priorities of these providers.

For the sake of example, remove configuration source like in the example below, so the default sources would not confuse you when adding your own.

```csharp
var builder = WebApplication.CreateBuilder(args);
builder.Configuration.Sources.Clear();
```

Multiple configuration sources can be used simultaneously. Values from all sources are stored in a unified key-value list. When same key is defined in multiple sources, the latest added source value overrides value from earlier sources.

To quickly check configuration during the debug:

```csharp
var configurationRoot = app.Configuration as IConfigurationRoot;
if (configurationRoot != null)
{
    string debugView = configurationRoot.GetDebugView();
    Console.WriteLine("Configuration debug view:");
    Console.WriteLine(debugView);
}
```

### Files

Using files for configuration is a common pattern for configuration. Most of frameworks will have such option readily available.

When to use:
- You are getting started and want a solution that is easy to manage.
- These are default config values that you want to share with other engineers.
- You want to use different set (file) of values based on some parameter.

Beware of:
- You might need to use different than default values during the development, so make sure that no one commits their WIP local values.
- Config files will be versioned together with other repo files, so be careful not to commit credentials.

#### Files configuration in ASP.NET

By default, new ASP.NET projects come prepared with `appsettings.json`, where you can specify your configurations in JSON.

This configuration should be enabled by default, but alternatively you can add it like this:

```csharp
builder.Configuration.AddJsonFile("appsettings.json");
```

### Environment variables

Environment variable configuration leverages OS environment variables. This provides more flexibility in some cases at a cost of complexity.

When to use:
- When you need mechanism to provide configuration values that you can change during the runtime and using different deployment models.

Beware of:
- Since environment variables are somewhat global variables, beware that some unrelated values that you don't necessarily expect can sneak in.

To add environment variables as a source of configuration:

```csharp
builder.Configuration.AddEnvironmentVariables();
```

Other configuration types allow variable nesting, which is not obviously translatable to key-value store like environment variables. For this a separator of 2 underscores (`__`) is used.

JSON configuration that looks like this:
```json
{
    "Database": {
        "Host": "localhost"
    }
}
```

Would be expressed equivalently like this in an environment variable:
```sh
Database__Host=localhost
```

### User secrets

Users secrets is an ASP.NET specific term, but similar mechanisms exist in other platforms as well, but might be called differently. User secrets stores configuration variables in some file stored outside of repository.

When to use:
- You want to have special config values for local development and you want to be sure that they will not get committed to shared repo.

Beware of:
- Will not work for deployments.

Usage:

```csharp
builder.Configuration.AddUserSecrets<Program>();
```

For user secrets to work a line like this will have to be added (or the IDE will add it automatically) to a `.csproj` file of the type's provided as generic argument assembly. It would be the assembly of `Program` class for the example above.

```xml
  <PropertyGroup>
    <UserSecretsId>f827f785-4194-4c6e-a24a-771ae0297199</UserSecretsId>
  </PropertyGroup>
```

Then the actual secrets will be stored in a file located in:
- Linux/Mac: `~/.microsoft/usersecrets/<user_secrets_id>/secrets.json`,
- Windows: `%APPDATA%\Microsoft\UserSecrets\<user_secrets_id>\secrets.json`.

Most IDEs will provide functionality to deal with user secrets out of the box or there will be an extension available for this. Otherwise they can be manipulated via `dotnet` CLI as described [here](https://learn.microsoft.com/en-us/aspnet/core/security/app-secrets).

## Additional reading

- https://sre.google/workbook/configuration-design/
- https://sre.google/workbook/configuration-specifics/